In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import datetime

In [2]:
PATH = "/Users/matthewkurkemelis/desktop/chromedriver"

In [251]:
# "https://www.espn.com/mens-college-basketball/boxscore?gameId=401263376" - Mich/IU
# "https://www.espn.com/mens-college-basketball/boxscore?gameId=401263380" - Iowa/Ohio st
# "https://www.espn.com/mens-college-basketball/boxscore?gameId=401260006" - Nova/BCdriver = selenium.webdriver.Chrome(PATH)
driver = selenium.webdriver.Chrome(PATH)
driver.get("https://www.espn.com/mens-college-basketball/boxscore?gameId=401260006")
print(driver.title)

team_records = driver.find_elements_by_class_name("record")
team_long_names = driver.find_elements_by_class_name("long-name")
team_ranks = driver.find_elements_by_class_name("rank")


for i in range(len(team_long_names)):
    print(team_records[i].text, team_long_names[i].text)
    
team_info = driver.find_elements_by_class_name("team-info")
team_info_list = []
for i in range(len(team_info)):
    team_info_list.append(team_info[i].text)
    
starter_header = driver.find_elements_by_xpath('//div[@class="col column-one gamepackage-away-wrap"]//table[@class="mod-data"]/thead[1]')
bench_header = driver.find_elements_by_xpath('//div[@class="col column-one gamepackage-away-wrap"]//table[@class="mod-data"]/thead[2]')
    
away_players = driver.find_elements_by_xpath('//div[@class="col column-one gamepackage-away-wrap"]//table[@class="mod-data"]/tbody/tr')
home_players = driver.find_elements_by_xpath('//div[@class="col column-two gamepackage-home-wrap"]//table[@class="mod-data"]/tbody/tr')

headers_list = []
for i in range(len(starter_header)):
    headers_list.append(starter_header[i].text)

away_box_score = []
for _ in range(len(away_players)):
    away_box_score.append(away_players[_].text)

home_box_score = []
for _ in range(len(home_players)):
    home_box_score.append(home_players[_].text)

driver.quit()

Villanova vs. Boston College - Box Score - November 25, 2020 - ESPN
1-0 Villanova
0-1 Boston College


In [252]:
team_info_list

['3\nVillanova\n1-0', 'Boston College\n0-1']

In [253]:
# add position after starter column name
for i in range(len(headers_list)):
    headers_list[i] = headers_list[i].split()
    headers_list[i][0] = "PLAYERS"
    headers_list[i].insert(1, "POS")
    headers_list[i].insert(4, "FGA")
    headers_list[i].insert(6, "3PTA")
    headers_list[i].insert(8, "FTA")

In [254]:
headers_list[0]

['PLAYERS',
 'POS',
 'MIN',
 'FG',
 'FGA',
 '3PT',
 '3PTA',
 'FT',
 'FTA',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TO',
 'PF',
 'PTS']

In [255]:
away_box_score

['J. Robinson-Earl\nF 26 9-14 0-0 0-0 6 4 10 1 1 2 2 3 18',
 'J. Samuels\nF 28 3-6 1-1 0-3 1 8 9 1 0 1 1 4 7',
 'J. Moore\nG 36 5-10 2-6 2-3 0 0 0 1 0 0 1 4 14',
 'C. Gillespie\nG 38 5-16 1-6 4-4 1 2 3 7 1 0 1 0 15',
 'C. Daniels\nG 36 5-9 2-6 2-2 0 3 3 1 1 0 1 3 14',
 'E. Dixon\nF 4 0-0 0-0 0-0 0 1 1 0 0 0 2 1 0',
 'C. Swider\nF 16 2-4 2-4 0-0 1 4 5 1 0 0 0 1 6',
 'B. Slater\nF 16 1-1 0-0 0-0 0 1 1 0 0 0 0 1 2',
 'TEAM 30-60 8-23 8-12 9 25 34 12 3 3 8 17 76',
 '  50.0% 34.8% 66.7%']

In [256]:
away_team_totals = away_box_score[-2:]

In [257]:
away_box_score = away_box_score[:-2]

In [258]:
away_box_score

['J. Robinson-Earl\nF 26 9-14 0-0 0-0 6 4 10 1 1 2 2 3 18',
 'J. Samuels\nF 28 3-6 1-1 0-3 1 8 9 1 0 1 1 4 7',
 'J. Moore\nG 36 5-10 2-6 2-3 0 0 0 1 0 0 1 4 14',
 'C. Gillespie\nG 38 5-16 1-6 4-4 1 2 3 7 1 0 1 0 15',
 'C. Daniels\nG 36 5-9 2-6 2-2 0 3 3 1 1 0 1 3 14',
 'E. Dixon\nF 4 0-0 0-0 0-0 0 1 1 0 0 0 2 1 0',
 'C. Swider\nF 16 2-4 2-4 0-0 1 4 5 1 0 0 0 1 6',
 'B. Slater\nF 16 1-1 0-0 0-0 0 1 1 0 0 0 0 1 2']

In [259]:
# convert web scraped box score data into usable format
# should separate FG, 3PT, and FT into made and attempted FG, 3PT, and FT
def convert_boxscore(box_score_list):
    """The only parameter is box_score_list, which is the
       box score data that was web scraped from the internet.
       box_score_list is a list of strings that contain the box
       score data.
       
       The function will alter the input and return nothing. 
    """
    # loop goes through each string of box_score_list and seperates each value by 
    for i in range(len(box_score_list)):
        player_name = [box_score_list[i].split("\n")[0]]
        rest_of_boxscore_str = box_score_list[i].split("\n")[1].split("-")
        rest_of_boxscore_list = []
        for j in rest_of_boxscore_str:
            rest_of_boxscore_list += j.split()
        box_score_list[i] = player_name + rest_of_boxscore_list
            

In [260]:
convert_boxscore(away_box_score)

In [261]:
away_box_score

[['J. Robinson-Earl',
  'F',
  '26',
  '9',
  '14',
  '0',
  '0',
  '0',
  '0',
  '6',
  '4',
  '10',
  '1',
  '1',
  '2',
  '2',
  '3',
  '18'],
 ['J. Samuels',
  'F',
  '28',
  '3',
  '6',
  '1',
  '1',
  '0',
  '3',
  '1',
  '8',
  '9',
  '1',
  '0',
  '1',
  '1',
  '4',
  '7'],
 ['J. Moore',
  'G',
  '36',
  '5',
  '10',
  '2',
  '6',
  '2',
  '3',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '4',
  '14'],
 ['C. Gillespie',
  'G',
  '38',
  '5',
  '16',
  '1',
  '6',
  '4',
  '4',
  '1',
  '2',
  '3',
  '7',
  '1',
  '0',
  '1',
  '0',
  '15'],
 ['C. Daniels',
  'G',
  '36',
  '5',
  '9',
  '2',
  '6',
  '2',
  '2',
  '0',
  '3',
  '3',
  '1',
  '1',
  '0',
  '1',
  '3',
  '14'],
 ['E. Dixon',
  'F',
  '4',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '2',
  '1',
  '0'],
 ['C. Swider',
  'F',
  '16',
  '2',
  '4',
  '2',
  '4',
  '0',
  '0',
  '1',
  '4',
  '5',
  '1',
  '0',
  '0',
  '0',
  '1',
  '6'],
 ['B. Slater',
  'F',
  '16',
 

In [262]:
home_team_totals = home_box_score[-2:]

In [263]:
home_team_totals

['TEAM 26-55 8-26 7-11 7 24 31 10 6 9 11 16 67', '  47.3% 30.8% 63.6%']

In [264]:
home_box_score = home_box_score[:-2]

In [265]:
home_box_score

['C. Felder\nF 28 3-4 1-2 0-0 2 2 4 0 2 4 0 0 7',
 'S. Mitchell\nF 37 0-2 0-1 3-4 2 10 12 0 2 4 0 1 3',
 'J. Heath\nG 33 7-14 2-8 0-0 1 3 4 2 0 1 0 2 16',
 'W. Tabbs\nG 21 4-13 2-5 1-1 0 1 1 0 0 0 2 4 11',
 'M. Ashton-Langford\nG 27 6-11 2-5 1-2 1 5 6 3 1 0 1 3 15',
 'K. Williams\nF 5 0-0 0-0 0-0 0 1 1 0 0 0 0 0 0',
 'F. Scott\nF 16 3-5 0-1 0-1 0 1 1 1 0 0 2 1 6',
 'J. Vander Baan\nC 0 0-0 0-0 0-0 0 0 0 0 0 0 0 0 0',
 'D. Langford Jr.\nG 9 0-0 0-0 0-0 0 1 1 2 1 0 2 2 0',
 'R. Kelly\nG 24 3-6 1-4 2-3 0 0 0 2 0 0 3 3 9']

In [266]:
convert_boxscore(home_box_score)

In [267]:
home_box_score

[['C. Felder',
  'F',
  '28',
  '3',
  '4',
  '1',
  '2',
  '0',
  '0',
  '2',
  '2',
  '4',
  '0',
  '2',
  '4',
  '0',
  '0',
  '7'],
 ['S. Mitchell',
  'F',
  '37',
  '0',
  '2',
  '0',
  '1',
  '3',
  '4',
  '2',
  '10',
  '12',
  '0',
  '2',
  '4',
  '0',
  '1',
  '3'],
 ['J. Heath',
  'G',
  '33',
  '7',
  '14',
  '2',
  '8',
  '0',
  '0',
  '1',
  '3',
  '4',
  '2',
  '0',
  '1',
  '0',
  '2',
  '16'],
 ['W. Tabbs',
  'G',
  '21',
  '4',
  '13',
  '2',
  '5',
  '1',
  '1',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '2',
  '4',
  '11'],
 ['M. Ashton-Langford',
  'G',
  '27',
  '6',
  '11',
  '2',
  '5',
  '1',
  '2',
  '1',
  '5',
  '6',
  '3',
  '1',
  '0',
  '1',
  '3',
  '15'],
 ['K. Williams',
  'F',
  '5',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['F. Scott',
  'F',
  '16',
  '3',
  '5',
  '0',
  '1',
  '0',
  '1',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '2',
  '1',
  '6'],
 ['J. Vander Baan',
  'C',
  '0